<a href="https://colab.research.google.com/github/rahiakela/small-language-models-fine-tuning/blob/main/domain-specific-small-language-models/01-tuning-for-specific-domain/02_fine_tuning_bert_for_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tuning BERT for Question Answering


The code in this notebook is to show how to fine tune a [DistilBert](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) model for extractive question answering (extract the answer from a given context). While all the steps refer to DistilBert, the same apply to a larger pool of LLM architectures.   
While no hardware acceleration is required to execute all the code cells, it is recommended to use a GPU to speed up the fine tuning step.

## Settings

Install the missing requirements in the Colab VM (HF's Datasets and Accelerate).

In [ ]:
!pip install datasets accelerate

## Data Preparation

Load a subset (5000 samples) of the *SQuAD* dataset using the HF's *Datasets* library.

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

Split the dataset's `train` split into a train and test set (80%/20%) with the *train_test_split* method:

In [3]:
squad = squad.train_test_split(test_size=0.2)

Display a sample:

In [4]:
squad["train"][0]

{'id': '56ce304daab44d1400b88510',
 'title': 'New_York_City',
 'context': 'New York—often called New York City or the City of New York to distinguish it from the State of New York, of which it is a part—is the most populous city in the United States and the center of the New York metropolitan area, the premier gateway for legal immigration to the United States and one of the most populous urban agglomerations in the world. A global power city, New York exerts a significant impact upon commerce, finance, media, art, fashion, research, technology, education, and entertainment, its fast pace defining the term New York minute. Home to the headquarters of the United Nations, New York is an important center for international diplomacy and has been described as the cultural and financial capital of the world.',
 'question': 'What city has been called the cultural capital of the world?',
 'answers': {'text': ['New York'], 'answer_start': [0]}}

The three important fields are:

- `answers`: the starting location of the answer token and the answer text.
- `context`: background information from which the model needs to extract the answer.
- `question`: the question a model should answer.

Load a DistilBERT tokenizer to process the `question` and `context` fields in the training set.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Some data preprocessing is needed:

1. Some examples in a dataset may have a very long `context` that exceeds the maximum input length of the model. To deal with longer sequences, truncate only the `context` by setting `truncation="only_second"`.
2. Map the start and end positions of the answer to the original `context` by setting
   `return_offset_mapping=True`.
3. Use the *sequence_ids* method to
   find which part of the offset corresponds to the `question` and which corresponds to the `context`.

Let's define a function that implementes the aforementioned preprocessing steps:

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

To apply the preprocessing function over the entire dataset, we can use the HF's Datasets *map* function. By setting `batched=True`, multiple elements of the dataset will be processed at once.

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Now create a batch of examples using DefaultDataCollator. Unlike other data collators in the HF's Transformers library, it doesn't apply any additional preprocessing.

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Fine Tuning

Load DistilBERT with *AutoModelForQuestionAnswering*:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased", device_map="auto")

Define the training hyperparameters in a *TrainingArguments* instance. The only required parameter is *output_dir* which specifies where to save our model.

In [10]:
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
)

Pass the training arguments to a *Trainer* instance along with the model, the dataset, the tokenizer, and the data collator.

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

Start the fine tuning:

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.302944
2,2.731400,1.705511
3,2.731400,1.627654


TrainOutput(global_step=750, training_loss=2.2862471110026044, metrics={'train_runtime': 489.2193, 'train_samples_per_second': 24.529, 'train_steps_per_second': 1.533, 'total_flos': 1175877900288000.0, 'train_loss': 2.2862471110026044, 'epoch': 3.0})

## Inference

The finetuned model can now be used for inference. Let's provide a question and some context we'd like the model to predict:

In [13]:
question = "How many official league titles has Juventus won?"
context = """Juventus Football Club (from Latin: iuventūs), colloquially known as Juve,
is a professional football club based in Turin, Piedmont, Italy, that competes in the Serie A,
the top tier of the Italian football league system. Founded in 1897 by a group of Torinese students,
the club has worn a black and white striped home kit since 1903 and has played home matches
in different grounds around its city, the latest being the 41,507-capacity Juventus Stadium.
Nicknamed la Vecchia Signora (the Old Lady), the club has won 36 official league titles, 14 Coppa Italia
titles and nine Supercoppa Italiana titles, being the record holder for all these competitions;"""

First, tokenize the text and return PyTorch tensors:

In [14]:
import torch
from transformers import AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

inputs = tokenizer(question, context, return_tensors="pt")
inputs.to(device)

{'input_ids': tensor([[  101,  2129,  2116,  2880,  2223,  4486,  2038, 22760,  2180,  1029,
           102, 22760,  2374,  2252,  1006,  2013,  3763,  1024,  1045, 27346,
          5809,  1007,  1010, 23992,  2124,  2004, 18414,  3726,  1010,  2003,
          1037,  2658,  2374,  2252,  2241,  1999, 13667,  1010, 18873,  1010,
          3304,  1010,  2008, 14190,  1999,  1996,  8668,  1037,  1010,  1996,
          2327,  7563,  1997,  1996,  3059,  2374,  2223,  2291,  1012,  2631,
          1999,  6347,  2011,  1037,  2177,  1997, 23413, 14183,  2493,  1010,
          1996,  2252,  2038,  6247,  1037,  2304,  1998,  2317, 17983,  2188,
          8934,  2144,  5778,  1998,  2038,  2209,  2188,  3503,  1999,  2367,
          5286,  2105,  2049,  2103,  1010,  1996,  6745,  2108,  1996,  4601,
          1010,  2753,  2581,  1011,  3977, 22760,  3346,  1012,  9919,  2474,
          2310, 25955,  2050,  3696,  6525,  1006,  1996,  2214,  3203,  1007,
          1010,  1996,  2252,  2038,  

Then let's pass our inputs to the model and return the `logits`:

In [15]:
from transformers import AutoModelForQuestionAnswering

with torch.no_grad():
    outputs = model(**inputs)

Get the highest probability from the model output for the start and end positions:

In [16]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

Decode the predicted tokens to get the answer in natural language:

In [17]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'36'